# Differential Rotation

In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

In [ ]:
import starry
import numpy as np
import matplotlib.pyplot as plt

starry.config.lazy = False
starry.config.quiet = True

In [ ]:
map = starry.Map(ydeg=5, drorder=1)
map.alpha = 0.1

# This is a bit hacky: compute the Ylm expansion of cos(latitude)
# then rotate it 90 degrees so it's cos(longitude) instead.
# Expanding a longitudinal cosine directly doesn't work as well
map.load(np.tile(np.cos(np.linspace(-np.pi, np.pi, 500)), (1000, 1)).T)
map.rotate([0, 0, 1], 90)

In [ ]:
map.show(projection="rect")

In [ ]:
# Render the map at 5 phases
theta = np.linspace(0, 360 * 2, 9)
res = 300
images = map.render(projection="rect", theta=theta, res=res)
lat = np.linspace(-90, 90, res, endpoint=False)
lon = np.linspace(-180, 180, res, endpoint=False)

# Compute the longitude of the maximum on the side
# facing the observer; it's easiest if we just mask the far side
images_masked = np.array(images)
images_masked[:, :, : (res // 4)] = 0
images_masked[:, :, -(res // 4) :] = 0
lon_starry = [lon[np.argmax(img, axis=1)] for img in images_masked]

# Compute the expected longitude of the maximum given
# the linear differential rotation law
lon_exact = [-theta_i * map.alpha * np.sin(lat * np.pi / 180) ** 2 for theta_i in theta]

In [ ]:
# Plot it!
fig, ax = plt.subplots(3, 3, figsize=(12, 8))
ax = ax.flatten()
for k in range(9):
    ax[k].imshow(images[k], extent=(-180, 180, -90, 90), cmap="plasma")
    ax[k].plot(lon_exact[k], lat)
    ax[k].plot(lon_starry[k], lat)
    ax[k].set_title(r"$\theta = {}^\circ$".format(theta[k]))
    for tick in ax[k].xaxis.get_major_ticks() + ax[k].yaxis.get_major_ticks():
        tick.label.set_fontsize(8)